In [346]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
sns.set_theme(palette=sns.color_palette("Set2"),style="darkgrid")

In [347]:
data = pd.read_csv("IMDb Movies India.csv",encoding='latin-1')

In [348]:
data.head(10)

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
5,...Aur Pyaar Ho Gaya,(1997),147 min,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
6,...Yahaan,(2005),142 min,"Drama, Romance, War",7.4,"1,086",Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
7,.in for Motion,(2008),59 min,Documentary,NaN,NaN,Anirban Datta,NaN,NaN,NaN
8,?: A Question Mark,(2012),82 min,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia
9,@Andheri,(2014),116 min,"Action, Crime, Thriller",4.0,11,Biju Bhaskar Nair,Augustine,Fathima Babu,Byon


In [349]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [350]:
data.duplicated(subset=["Name"]).sum()

1671

In [351]:
data.isna().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [352]:
data= data.dropna(subset=["Year","Genre","Director","Actor 1","Actor 2","Actor 3","Rating"])

In [353]:
data.shape

(7558, 10)

In [354]:
data["Duration"] = data.loc[:,"Duration"].str.replace(" min","")
data["Duration"] = pd.to_numeric(data.loc[:,"Duration"])

In [355]:
data["Votes"] = data.loc[:,"Votes"].str.replace(",","")
data["Votes"] = pd.to_numeric(data.loc[:,"Votes"])

In [356]:
data["Duration"] = data.loc[:,'Duration'].fillna(data.groupby("Genre")["Duration"].transform('mean'))

In [357]:
data = data.dropna().reset_index(drop=True)

In [358]:
data.shape

(7516, 10)

In [359]:
data.duplicated(subset=["Name","Year"]).sum()

3

In [360]:
data=data.drop_duplicates(keep="first",subset=["Name","Year"]).reset_index(drop=True)

In [362]:
data.columns

Index(['Name', 'Year', 'Duration', 'Genre', 'Rating', 'Votes', 'Director',
       'Actor 1', 'Actor 2', 'Actor 3'],
      dtype='object')

In [370]:
data.Year.unique()

array(['(2019)', '(1997)', '(2005)', '(2012)', '(2014)', '(2004)',
       '(2016)', '(1991)', '(2018)', '(2010)', '(1958)', '(2021)',
       '(2017)', '(2009)', '(1993)', '(2002)', '(1946)', '(2008)',
       '(1994)', '(2007)', '(2013)', '(2003)', '(1998)', '(1979)',
       '(1974)', '(2015)', '(2006)', '(1981)', '(2020)', '(1985)',
       '(2011)', '(1988)', '(1995)', '(1987)', '(1999)', '(1973)',
       '(1968)', '(1953)', '(1986)', '(1982)', '(1977)', '(1950)',
       '(1969)', '(1948)', '(1967)', '(1970)', '(1990)', '(1989)',
       '(1947)', '(2001)', '(2000)', '(1971)', '(1978)', '(1944)',
       '(1963)', '(1992)', '(1976)', '(1984)', '(1975)', '(1980)',
       '(1966)', '(1972)', '(1956)', '(1960)', '(1964)', '(1952)',
       '(1959)', '(1951)', '(1954)', '(1962)', '(1961)', '(1957)',
       '(1965)', '(1996)', '(1933)', '(1955)', '(1983)', '(1936)',
       '(1949)', '(1940)', '(1945)', '(1938)', '(1941)', '(1942)',
       '(1932)', '(1935)', '(1937)', '(1931)', '(1943)', '(191